# Load packages

In [1]:
import Astroshaper
import GLMakie
import JLD2
import SPICE
import Downloads
import Statistics

# Download files

In [2]:
pwd()

"/Users/masanorikanamaru/Documents/GitHub/Astroshaper-examples/TPM_Ryugu"

In [4]:
path_obj = "SHAPE_SFM_49k_v20180804.obj"
if !isfile(path_obj)
    url_obj = "https://data.darts.isas.jaxa.jp/pub/hayabusa2/paper/Watanabe_2019/SHAPE_SFM_49k_v20180804.obj"
    Downloads.download(url_obj, path_obj)
end

In [194]:
# using Plots
# using FileIO
# using JLD2
# using Profile

In [3]:
using Revise
using Astroshaper

In [4]:
ENV["COLUMNS"] = 1000
ENV["LINES"] = 20;

In [5]:
shapedir = "/Users/masanorikanamaru/Documents/shape/ryugu"

shapename = "ryugu_v252_f500.obj"
# shapename = "ryugu_v752_f1500.obj"
# shapename = "ryugu_test.obj"

# shapename = "SHAPE_SPC_49k_v20190802.obj"
# shapename = "SHAPE_SPC_200k_v20190802.obj"
# shapename = "SHAPE_SPC_800k_v20190802.obj"
# shapename = "SHAPE_SPC_3M_v20190802.obj"

# shapename = "SHAPE_SFM_49k_v20180804.obj"
# shapename = "SHAPE_SPC_49k_v20200323.obj"
# shapename = "SHAPE_SPC_3M_v20200323.obj"

shapepath = joinpath(shapedir, shapename)
@show shapepath;

shapepath = "/Users/masanorikanamaru/Documents/shape/ryugu/ryugu_v252_f500.obj"


In [6]:
# shapepath = splitext(shapepath)[1] * ".jld2"

In [7]:
shape = ShapeModel(shapepath; scale=1000, find_visible_facets=true, save_shape=false)

Shape model
-----------
Nodes             : 252
Faces             : 500
Surface area      : 2.6740222482370236e6
Volume            : 3.759699719284749e8
Equivalent radius : 447.7354864412234
Center-of-Figure  : [0.37115829680556717, 0.14301318037402377, 0.02972401268611568]
Inertia tensor    : 
    | Ixx Ixy Ixz |   [0.0, 0.0, 0.0]
    | Iyx Iyy Iyz | = [0.0, 0.0, 0.0]
    | Izx Izy Izz |   [0.0, 0.0, 0.0]


In [8]:
# draw(shape)
# draw(shape, data=:radius)
# draw(shape, data=:temperature)
# draw(shape; data=:illumination, r̂☉=[1,0,0.])
# draw(shape, data=[center[3] for center in shape.facets.center])

In [15]:
orbit = OrbitalElements(RYUGU)

--------------------
  Orbital elements  
--------------------
    Semi-mojor axis         : a  = 1.18956373 [AU]
    Eccentricity            : e  = 0.19027921 [-]
    Lon. of ascending node  : Ω  = 251.589203 [deg]
    Argument of periapsis   : ω  = 211.435963 [deg]
    Inclination             : I  = 5.8840222 [deg]
    Periapsis passage time  : tₚ = -2.4948038004043424e6 [sec]
    Mean anomaly            : Φ  = 21.9353799 [deg]
--------------------
  Other parameters  
--------------------
    Gravitational parameter : μ = 1.32712440018e20 [m^3/s^2]
    Mean motion             : n = 0.7596656791419168 [deg/day]
    Orbital period          : P = 473.8926739544681 [day]
------------------
  Time-variables  
------------------
    Time                    : t  = 0.0 [sec]
    Eccentric anomaly       : u  = 26.861352366162944 [deg]
    True anomaly            : ν  = 32.29438904892344 [deg]
    Position                : r  = [1.2489380713478127e11, 7.893746967835626e10, 0.0] [m]
    Veloci

In [10]:
spin = SpinParams(RYUGU, orbit)

-------------------
  Spin parameters  
-------------------
Right ascension (RA) : α = 96.4 [deg]
Declination (Dec)    : δ = -66.4 [deg]
Ecliptic longitude   : λ = 179.33757188938182 [deg]
Ecliptic latitude    : β = -87.44207056697601 [deg]
Obliquity            : ε = 171.65664649001607 [deg]
Spin period          : P = 7.63262 [hours]
Spin rate            : ω = 0.00022866712242903872 [rad/sec]
Vernal equinox lon.  : γ = 154.3323208882217 [deg]
                           (longitude from the periheion direction)
Time                 : t  = 0.0 [sec]
Initial spin phase   : ϕ₀ = 0.0 [deg]
Spin phase           : ϕ  = 0.0 [deg]


# Thermophysical Model

In [11]:
thermo_params = ThermoParams(
    A_B   = 0.04,
    A_TH  = 0.0,
    k     = 0.1,
    ρ     = 1270.0,
    Cp    = 600.0,
    ϵ     = 1.,
    t_bgn = 0.0,
    t_end = spin.P * 10,
    Nt    = 721,
    z_max = 0.6,
    Nz    = 41,
    P     = spin.P,
);

In [13]:
# savepath = "tmp.jld2"
@time timestamp = run_TPM(shape, orbit, spin, thermo_params)

  0.280710 seconds (140.58 k allocations: 12.344 MiB)


,t,u,ν,ϕ,f_x,f_y,f_z,τ_x,τ_y,τ_z,E_in,E_out,E_cons,Ē_cons
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.468819,0.563643,0.0,9.92025e-6,4.44963e-6,6.01218e-7,-0.000416175,0.000149287,-0.000183821,8.38911e8,0.0,0.0,0.0
2,381.631,0.46889,0.563727,0.0872665,0.0251434,0.0132682,-0.00181507,0.37064,-0.618929,-0.19855,8.3566e8,1.68572e7,0.0201723,0.0100862
3,763.262,0.46896,0.56381,0.174533,0.0510587,0.025989,-0.00369733,0.70619,-1.15349,-0.382106,8.34888e8,3.4082e7,0.0408222,0.0203315
4,1144.89,0.469031,0.563894,0.261799,0.0787493,0.0381852,-0.00567068,1.03461,-1.65732,-0.55864,8.35013e8,5.22418e7,0.062564,0.0308896
5,1526.52,0.469101,0.563977,0.349066,0.106405,0.0487858,-0.00763911,1.35364,-2.10314,-0.708673,8.34828e8,7.00753e7,0.0839398,0.0414997
6,1908.15,0.469172,0.56406,0.436332,0.133083,0.057346,-0.00956968,1.67198,-2.49911,-0.831672,8.33457e8,8.69602e7,0.104337,0.0519725
7,2289.79,0.469242,0.564144,0.523599,0.158207,0.0637697,-0.0114638,1.99403,-2.86807,-0.930422,8.38661e8,1.02548e8,0.122276,0.0620158
8,2671.42,0.469313,0.564227,0.610865,0.181472,0.0679746,-0.0133198,2.29748,-3.2293,-1.03886,8.41196e8,1.16765e8,0.138809,0.071615
9,3053.05,0.469383,0.564311,0.698132,0.202615,0.0702689,-0.0151363,2.57798,-3.60047,-1.1569,8.43099e8,1.29508e8,0.153609,0.0807254


In [14]:
Astroshaper.describe(timestamp)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,t,1.37387e5,0.0,1.37387e5,2.74774e5,0,Float64
2,u,0.494167,0.468819,0.494179,0.519469,0,Float64
3,ν,0.593563,0.563643,0.59359,0.623373,0,Float64
4,ϕ,31.4159,0.0,31.4159,62.8319,0,Float64
5,f_x,0.555033,9.92025e-6,0.588482,0.642349,0,Float64
6,f_y,0.152681,4.44963e-6,0.161908,0.218619,0,Float64
7,f_z,-0.0679187,-0.102933,-0.0737024,6.01218e-7,0,Float64
8,τ_x,2.37387,-1.18597,2.24983,6.10219,0,Float64
9,τ_y,-8.24232,-11.2672,-8.11157,0.000149287,0,Float64


In [15]:
# timestamp = Astroshaper.jldopen("tmp.jld2", "r") do file
#     file["timestamp"]
# end

In [16]:
C = 4.039541372643629e16
YORP = analyze_YORP(timestamp, spin, C)

@show shapename
@show YORP.τ̄
@show YORP.ω̇, YORP.ωε̇, YORP.ωψ̇
@show YORP_timescale(YORP.ω̇, 3.5, 7.63262);  # 3.5時間から7.6時間まで減速する時間スケール [yr]

shapename = "ryugu_v252_f500.obj"
YORP.τ̄ = [2.3738664859828864, -8.242316607010164, -0.7373578467947709]
(YORP.ω̇, YORP.ωε̇, YORP.ωψ̇) = (-2.1094055215113005e-6, 6.81872181819457e-5, 6.045574839209872e-5)
YORP_timescale(YORP.ω̇, 3.5, 7.63262) = 1.7347912669031378e6


In [ ]:
function analyze_YORP(df, spin, MOI)
    τ̄ = [mean(df.τ_x), mean(df.τ_y), mean(df.τ_z)]

    τ̄_ω = τ̄ ⋅ spin.ŝ
    τ̄_ε = τ̄ ⋅ spin_perp_unit1(spin)
    τ̄_ψ = τ̄ ⋅ spin_perp_unit2(spin)

    ## [rad/sec/sec]
    ω̇  = τ̄_ω / MOI
    ωε̇ = τ̄_ε / MOI
    ωψ̇ = τ̄_ψ / MOI

    ## [deg/day/day]
    ω̇  = rad2deg(ω̇)  * (3600*24)^2
    ωε̇ = rad2deg(ωε̇) * (3600*24)^2
    ωψ̇ = rad2deg(ωψ̇) * (3600*24)^2

    (τ̄ = τ̄, τ̄_ω = τ̄_ω, τ̄_ε = τ̄_ε, τ̄_ψ = τ̄_ψ, ω̇ = ω̇, ωε̇ = ωε̇, ωψ̇ = ωψ̇)
end

# YORP効果 @現在の軌道

In [17]:
thermo_params = ThermoParams(
    A_B   = 0.04,
    # A_B   = fill(0.04, shape.num_face),
    A_TH  = 0.0,
    k     = 0.1,
    # k     = fill(0.1, shape.num_face),
    ρ     = 1270.0,
    Cp    = 600.0,
    ϵ     = 1.,
    t_bgn = 0.0,
    t_end = spin.P * 10,
    Nt    = 721,
    z_max = 0.6,
    Nz    = 41,
    P     = spin.P,
);

In [18]:
@time τ̄ = run_YORP(shape, orbit, spin, thermo_params)

  0.198256 seconds (2 allocations: 160 bytes)


3-element Vector{Float64}:
  2.373866485982889
 -8.242316607010173
 -0.737357846794772

In [19]:
C = 4.039541372643629e16
ω̇, ωε̇, ωψ̇ = torque2rate(τ̄, spin, C)  # [deg/day²]

@show shapename
@show τ̄
@show ω̇, ωε̇, ωψ̇
@show YORP_timescale(3.5, 7.63262, ω̇);  # 3.5時間から7.6時間まで減速する時間スケール [yr]

shapename = "ryugu_v252_f500.obj"
τ̄ = [2.373866485982889, -8.242316607010173, -0.737357846794772]
(ω̇, ωε̇, ωψ̇) = (-2.109405521511298e-6, 6.818721818194578e-5, 6.045574839209878e-5)
YORP_timescale(3.5, 7.63262, ω̇) = 1.7347912669031401e6


In [15]:
##############################
#  　ryugu_v252_f500.obj     #
#　  t_end = orbit.T         #
##############################

# shapename = "ryugu_v252_f500.obj"
# τ̄ = [2.102139762963553, -1.9503690549955461, 0.10649756611585151]
# (ω̇, ωε̇, ωψ̇) = (-2.417553631012555e-6, 8.713420516412938e-6, 2.9006181145065934e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 1.514706707465591

##############################
#　  ryugu_test.obj          #
#　  t_end = spin.P * 10     #
##############################

# shapename = "ryugu_test.obj"
# τ̄ = [2.5306956279461983, -8.233792225809143, -0.7075406040546186]
# (ω̇, ωε̇, ωψ̇) = (-2.305600098551757e-6, 6.734927363385765e-5, 6.191331835096895e-5)
# YORP_timescale(3.5, 7.63262, ω̇) = 1.5871695526790055e6

In [ ]:
# open("ryugu_test_temps.txt","w") do f
#     for smesh in shape.smeshes
#         println(f, smesh.Tz[begin], ", ", smesh.flux.sun, ", ", smesh.flux.scat, ", ", smesh.flux.rad)
#     end
# end

In [ ]:
# shapename = "ryugu_v252_f500.obj"

# Thermophysical parameters
# -------------------------
# A_B   : 0.04
# A_TH  : 0.0
# k     : 0.1
# ρ     : 1270.0
# Cₚ    : 600.0
# ϵ     : 1.0
# P     : 27477.432
# l     : 0.21287051812296282
# Γ     : 276.04347483684523
# Δt    : 0.01388888888888889
# t_bgn : 0.0
# t_end : 1490.1074827395094
# Nt    : 107288
# Δz    : 0.09395382778392625
# z_max : 2.8186148335177874
# Nz    : 31
# λ     : 0.12520702099737538

# 影 + 熱伝導
# τ̄ = [2.166292988343463, -1.8232052646633534, 0.12149914462601369]
# (ω̇, ωε̇, ωψ̇) = (-2.3559241586480615e-6, 7.198547956478111e-6, 2.903520943282301e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 1.5543304681988876

# 影 + 熱伝導 + 放射の再吸収
# τ̄ = [2.1867628755835007, -1.9130691894522611, 0.20980627031427984]
# (ω̇, ωε̇, ωψ̇) = (-3.3918521233894786e-6, 7.818508184547549e-6, 2.964269717748932e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 1.0796121314667426

# 影 + 熱伝導 + 放射の再吸収 + 散乱光による自己加熱
# τ̄ = [2.1865019814933127, -1.9132227674055484, 0.20659523329863488]
# (ω̇, ωε̇, ωψ̇) = (-3.358599466983103e-6, 7.826075560400724e-6, 2.9640911737278277e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 1.0903011021560833

# 影 + 熱伝導 + 放射の再吸収 + 散乱光による自己加熱 + 熱放射の再吸収による自己加熱
# τ̄ = [2.1769952611580217, -1.9075575309460082, 0.11690136500927037]
# (ω̇, ωε̇, ωψ̇) = (-2.4174445056821524e-6, 7.953525830274134e-6, 2.9524204264352846e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 1.5147750825077182

In [ ]:
# shapename = "ryugu_v752_f1500.obj"

# 影
# τ̄ = [3.3097974041391374, -1.7111464770712586, 0.12728322411992796]
# (ω̇, ωε̇, ωψ̇) = (-1.5003553780529698e-6, 9.426867138482173e-7, 3.9434066533743185e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 2.440678224718057

# 影 + 熱伝導 + 放射の再吸収 + 散乱光による自己加熱 + 熱放射の再吸収による自己加熱
# τ̄ = [2.165370019439715, -1.8680106417353224, 0.027423863292429056]
# (ω̇, ωε̇, ωψ̇) = (-1.433047521048842e-6, 7.77033687219021e-6, 2.923188963996669e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 2.5553128188466436

In [ ]:
# shapename = "ryugu_test.obj"

# 影
# τ̄ = [3.2623680861875415, -1.7632177354631766, 0.2112963317251145]
# (ω̇, ωε̇, ωψ̇) = (-2.484150619384357e-6, 1.5205017761112165e-6, 3.9220246601817255e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 1.4740993045985396

# 影 + 熱伝導 + 放射の再吸収 + 散乱光による自己加熱 + 熱放射の再吸収による自己加熱
# τ̄ = [2.1336218786667724, -1.878807112180092, 0.04045730505339008]
# (ω̇, ωε̇, ωψ̇) = (-1.6056650245345357e-6, 7.99631943219241e-6, 2.8978423396048164e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 2.280603142373392